In [771]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [772]:
# using the Apple metal framework to move tensors to GPU
torch.backends.mps.is_available()

True

In [773]:
with open('names.txt','r') as w:
    names = w.read()
    names = names.splitlines()

In [774]:
names[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [775]:
len(names)

32033

In [776]:
chars = sorted(list(set(''.join(names))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos= {s:i for i,s in stoi.items()}
print(stoi)
print(itos)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [777]:
block_size = 3

x, y = [], []

for w in names:
    
    # print(w)

    context = [0] * block_size

    for ch in w + '.':
        ix = stoi[ch]
        x.append(context)
        y.append(ix)

        # print(''.join(itos[i] for i in context),'--->',itos[ix])

        context = context[1:] + [ix]

x = torch.tensor(x)
y = torch.tensor(y)

In [778]:
x[:4],y[:4]

(tensor([[ 0,  0,  0],
         [ 0,  0,  5],
         [ 0,  5, 13],
         [ 5, 13, 13]]),
 tensor([ 5, 13, 13,  1]))

Now we embedd our 27 characters into a 2-D space.

In [779]:
C = torch.randn((27,2))

### Below we do some tensor gymnastics to see how effective pytorch indexing is and how we can simply index into C from x using C[x].

In [780]:
x[:2], x[:2].shape

(tensor([[0, 0, 0],
         [0, 0, 5]]),
 torch.Size([2, 3]))

In [781]:
x[13,2]

tensor(1)

In [782]:
C[x[13]][2]

tensor([ 1.2569, -0.1743])

In [783]:
C[1]

tensor([ 1.2569, -0.1743])

In [784]:
C[[1,2,3]]

tensor([[ 1.2569, -0.1743],
        [ 0.3862, -0.5357],
        [-0.3077, -1.0447]])

In [785]:
C[x[:2]], C[x[:2]].shape

(tensor([[[ 1.5709,  0.8534],
          [ 1.5709,  0.8534],
          [ 1.5709,  0.8534]],
 
         [[ 1.5709,  0.8534],
          [ 1.5709,  0.8534],
          [ 1.2772, -1.2784]]]),
 torch.Size([2, 3, 2]))

## Now we implement the embedding lookup table

In [786]:
emb = C[x]
emb.shape

torch.Size([228146, 3, 2])

In [787]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [788]:
W1.shape, b1.shape

(torch.Size([6, 100]), torch.Size([100]))

In [789]:
emb.size()

torch.Size([228146, 3, 2])

## To move our inputs to the next layer we would want to do 
1. emb @ W1 + b
2. But, we can't do that as emb and W1 are not shape compatible
3. So we do something familiar, i.e reshape/view of emb

In [790]:
# h = emb.view(32,6) @ W1 + b1 we change this in favour of below, as the magic -1 pops up.
h = torch.tanh(emb.view(-1,6) @ W1 + b1) #also add a tanh to ensure ouputs btw -1 and 1, our non-linearity
h.shape

torch.Size([228146, 100])

In [791]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [792]:
logits = h @ W2 + b2

In [793]:
logits.shape

torch.Size([228146, 27])

In [794]:
counts = logits.exp()

In [795]:
probs = counts / counts.sum (1, keepdim = True)

In [796]:
probs.shape

torch.Size([228146, 27])

In [797]:
#  again why log, if the probability is very high, we get the output close to zero, else if close to zero, we get hugely -ve.
loss = -probs[torch.arange(32),y].log().mean()
loss

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32], [228146]

## Lets make it respectable :)

In [798]:
x.shape, y.shape #dataset

(torch.Size([228146, 3]), torch.Size([228146]))

In [806]:
# intialising all our embeddings, layers and biases
g = torch.Generator().manual_seed(2147483647) 
C = torch.randn((27,2),generator=g)
W1 = torch.randn((6,100),generator=g)
b1 = torch.randn(100,generator=g)
W2 = torch.randn((100,27),generator=g)
b2 = torch.randn(27,generator=g)
parameters = [C,W1,b1,W2,b2]

In [807]:
z = []
for p in parameters:
    z.append(p.numel()) #numel returns the number of elements in the tensor
x,sum(z)

(tensor([[ 0,  0,  0],
         [ 0,  0,  5],
         [ 0,  5, 13],
         ...,
         [26, 26, 25],
         [26, 25, 26],
         [25, 26, 24]]),
 3481)

In [808]:
sum(p.numel() for p in parameters)

3481

In [802]:
# forward pass
emb = C[x]
h = torch.tanh(emb.view(-1,6) @ W1 + b1) # (32,100)
logits = h @ W2 + b2 # (32,27)

## The below is a manual implemenation of negative log-likelihood loss or cross entropy loss, highly inefficient
# counts = logits.exp()
# probs = counts / counts.sum(1, keepdim=True)
# loss = -probs[torch.arange(32),y].log().mean()

loss = F.cross_entropy(logits,y) # we can always replace the manual implementation of cross entropy with the direct funtion call, as its much faster
loss

tensor(19.5052)

In [809]:
# enable gradient tracking
for p in parameters:
    p.requires_grad = True

In [818]:
from tqdm import trange

for _ in range(10):

    # minibatch initialisation
    ix = torch.randint(0,x.shape[0],(32,))

    # forward pass
    emb = C[x[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2 
    loss = F.cross_entropy(logits,y[ix]) 
    print(loss.item())

    # backward pass
    for p in parameters:
        p.grad = None

    loss.backward()

    for p in parameters:
        p.data += -0.1 * p.grad
    
    

5.654982089996338
4.8186492919921875
4.190366268157959
4.9438910484313965
4.705173015594482
3.991664409637451
5.158417224884033
3.690796136856079
4.268844127655029
3.672621965408325


In [811]:
logits.max(1)

torch.return_types.max(
values=tensor([ 8.2985,  9.8879, 18.4876,  ..., 11.3038, 13.8946, 12.4195],
       grad_fn=<MaxBackward0>),
indices=tensor([ 9,  8,  9,  ..., 15, 11, 17]))

In [812]:
y

tensor([ 5, 13, 13,  ..., 26, 24,  0])